In [3]:
from nuscenes.nuscenes import NuScenes
from nuscenes.map_expansion.map_api import NuScenesMap
from nuscenes.eval.prediction.splits import get_prediction_challenge_split
from nuscenes.prediction import PredictHelper
from nuscenes.utils.splits import create_splits_scenes


DATA_PATH = "../data/v1.0-mini/" #path to data stored

nusc = NuScenes(version='v1.0-mini', dataroot=DATA_PATH, verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.5 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [9]:
import json

In [21]:
#         self.trainset = get_prediction_challenge_split("mini_train", dataroot=data_path)
helper = PredictHelper(nusc)

#get all the scenes
scenes = create_splits_scenes()
print(scenes.keys())
#get all the scenes in the trainset
set_name = "mini_train"
trainset = scenes[set_name] #List of scenes as part of training set
prediction_scenes = json.load(open(DATA_PATH+"maps/prediction_scenes.json", "r")) #Dictionary containing list of instance and sample tokens for each scene
 
print("Number of samples in train set: %d" % (len(trainset)))

dict_keys(['train', 'val', 'test', 'mini_train', 'mini_val', 'train_detect', 'train_track'])
Number of samples in train set: 8


In [30]:
idx = 1
scene = trainset[idx]
scene_tokens = prediction_scenes[scene] 

# print(scene_tokens)
if len(scene_tokens) < 2:
    print("Not enough agents in the scene")
    
else:
    #get the tokens in the scene: we will be using the instance tokens as that is the agent in the scene
    tokens = [scene_tok.split("_") for scene_tok in scene_tokens]
    print(tokens)
    instance_tokens, sample_tokens = list(list(zip(*tokens))[0]), list(list(zip(*tokens))[1]) #List of instance tokens and sample tokens

    print(instance_tokens)
    print("---------------")
    print(sample_tokens)

    # print(scenes["mini_train"])

[['d02ce7e39f534cb8b67ef101e32dbba7', '8bf071b1b92c4ff1bd5016bfb30de3f7'], ['d02ce7e39f534cb8b67ef101e32dbba7', 'b8766f4291d5449d95bab459ca724ad8'], ['d02ce7e39f534cb8b67ef101e32dbba7', 'dde297c8e12f44c4bc2f2c80e49d2b5a'], ['d02ce7e39f534cb8b67ef101e32dbba7', 'a0462df7186d4ca99ef58f455d53a42c'], ['d02ce7e39f534cb8b67ef101e32dbba7', 'd7bea9a3552b40e29d42324108dff575'], ['d02ce7e39f534cb8b67ef101e32dbba7', '3ffc4360d1084e6eae5067e87d79503f'], ['d02ce7e39f534cb8b67ef101e32dbba7', '05bc09f952ab4cf8b754405f20d503c5'], ['d02ce7e39f534cb8b67ef101e32dbba7', 'a860c02c06e54d9dbe83ce7b694f6c17'], ['d02ce7e39f534cb8b67ef101e32dbba7', '6873840fc1df4796a7b959e13e9fb78a'], ['813f2ab6ce964dce81bf47aadc0c854f', '27900b9c3cfb48ed8b49087098d4a15a'], ['813f2ab6ce964dce81bf47aadc0c854f', 'c4f9b75136384ec4a76c4adfc28b4259'], ['813f2ab6ce964dce81bf47aadc0c854f', 'a572dd2e95e94e4db66bb5dbfaf870b5'], ['9cbb3b5b8bf849599abcb0228eea3edd', 'b8766f4291d5449d95bab459ca724ad8'], ['9cbb3b5b8bf849599abcb0228eea3edd', 

In [29]:
def Extract(lst): 
    print(list(zip(*lst)))
    return list(list(zip(*lst))[0]) 
      
# Driver code 
lst = [[1, 2], [3, 4, 5], [6, 7, 8, 9]] 
print(Extract(lst)) 

[(1, 3, 6), (2, 4, 7)]
[1, 3, 6]
